# Introduction

This notebook will have a look at the large processed dataset (343 vars, 175,335 obs), and run a lasso regression to see which variables are most predictive of subjective wellbeing (measured on the Likert scale).

Before starting, these are the 15 variables I hand selected:

- gor_dv : Government Office Region
- urban_dv : Urban or rural area, derived
- nbrsnci_dv : Buckner's Neighbourhood Cohesion Instrument, short (α= .88)
- scghq1_dv : Subjective wellbeing (GHQ): Likert
- sex_dv : Sex, derived
- age_dv : Age, derived from dob_dv and intdat_dv
- ethn_dv : Ethnic group (derived from multiple sources)
- marstat_dv : Harmonised de facto marital status
- jbstat : Current economic activity
- qfhigh_dv : Highest educational qualification ever reported
- jbiindb_dv : Current job: Industrial classification (CNEF), two digits
- fimnnet_dv : total net personal income - no deductions
- fihhmnnet1_dv : total household net income - no deductions
- houscost1_dv : monthly housing cost including mortgage principal payments
- health : Long-standing illness or disability

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
%matplotlib inline
import pyreadstat

# Make 'head' scroll across the width of the screen
pd.set_option('display.max_columns', None)

In [2]:
# Import file

path = "/Users/arikatz/VSCode Projects/ukhls-informal-institutions-project/data/droppedvaraibleswithlotsofnegatives.dta"
df_full, meta = pyreadstat.read_dta(path)

print("Shape:", df_full.shape)
df_full.head()

Shape: (175335, 343)


,pidp,wave,wave_num,nbrsnci_dv,scghq1_dv,hidp,pno,hhorig,memorig,psu,strata,sampst,month,ivfio,ioutcome,sex,dvage,birthy,istrtdatd,istrtdatm,istrtdaty,lkmove,xpmove,jbstat,racel_dv,health,aidhh,aidxhh,j2has,bensta2,bensta3,bensta4,bensta5,bensta6,bensta7,bensta96,fiyrdia,finnow,finfut,vote1,vote6,mobuse,nch14resp,nch415resp,nchresp,nnatch,nadoptch,nchunder16,nch5to15,nch10to15,sclfsat1,sclfsat2,sclfsat7,sclfsato,marstat,employ,hgbiom,hgbiof,hgpart,respf16,respm16,intdatd_if,intdatm_if,intdaty_if,doby_if,age_if,pn1pno,pn2pno,pns1pno,pns2pno,hhsize,jbhas,istrtdathh,istrtdatmm,istrtdatss,ienddathh,ienddatmm,ienddatss,j2pay_if,fimngrs_tc,fimngrs_dv,fimnlabgrs_tc,fimnlabgrs_dv,fimnlabnet_tc,fimnlabnet_dv,fiyrinvinc_tc,fiyrinvinc_dv,fibenothr_tc,fibenothr_dv,j2pay_dv,j2paynet_dv,sex_dv,age_dv,intdatd_dv,intdatm_dv,intdaty_dv,doby_dv,pensioner_dv,npensioner_dv,marstat_dv,npn_dv,npns_dv,ngrp_dv,nnsib_dv,nnssib_dv,ethn_dv,fimnmisc_dv,fimnprben_dv,fimninvnet_dv,fimnpen_dv,fimnsben_dv,fimnnet_dv,country,gor_dv,urban_dv,hhresp_dv,xtra5min_dv,agegr5_dv,agegr10_dv,agegr13_dv,livesp_dv,cohab_dv,single_dv,mastat_dv,hhtype_dv,buno_dv,depchl_dv,nchild_dv,ndepchl_dv,respm16_dv,respf16_dv,rach16_dv,hrpid,hrpno,ppno,sppno,fnpno,fnspno,mnpno,mnspno,grfpno,grmpno,qfhigh_dv,qfhighfl_dv,hiqual_dv,jbiindb_dv,sf12pcs_dv,sf12mcs_dv,scflag_dv,paygu_if,paynu_if,seearngrs_if,fiyrinvinc_if,fibenothr_if,fimnlabgrs_if,fimngrs_if,ind5mus_xw,ivfho,intdated,intdatem,intdatey,ivh1,ivh2,ivh3,ivh4,ivh5,ivh6,ivh7,ivh8,ivh9,ivh10,ivh11,ivh12,ivh13,ivh14,ivh15,ivh16,hsbeds,hsrooms,hsownd,fuelhave1,fuelhave2,fuelhave3,fuelhave4,fuelhave96,fuelduel,heatch,xphsdct,xphsdba,cduse1,cduse2,cduse5,cduse6,cduse7,cduse8,cduse9,cduse12,cduse13,cduse96,pcnet,xpfood1_g3,xpfdout_g3,xpaltob_g3,ncars,hhintlang,n10to15,fihhmngrs_dv,fihhmngrs_tc,fihhmnlabgrs_dv,fihhmnlabgrs_tc,ctband_if,fihhmnnet1_dv,fihhmnlabnet_dv,fihhmnmisc_dv,fihhmnprben_dv,fihhmninv_dv,fihhmnpen_dv,fihhmnsben_dv,houscost1_dv,houscost2_dv,fihhmngrs1_dv,ctband_dv,ncouple_dv,nonepar_dv,nkids_dv,nch02_dv,nch34_dv,nch511_dv,nch1215_dv,npens_dv,nemp_dv,nue_dv,nwage_dv,nchoecd_dv,nadoecd_dv,ieqmoecd_dv,tenure_dv,fihhnegsei_if,fihhmngrs_if,issue_num,aintlen,outcome,ivtnc,w6osmflag,dcsedfl_dv,lwenum_dv,fwenum_dv,lwintvd_dv,fwintvd_dv,b_hidp,b_pno,b_ivfio,b_ivfho,b_month,c_hidp,c_pno,c_ivfio,c_ivfho,c_month,d_hidp,d_pno,d_ivfio,d_ivfho,d_month,e_hidp,e_pno,e_ivfio,e_ivfho,e_month,f_hidp,f_pno,f_ivfio,f_ivfho,f_month,g_hidp,g_pno,g_ivfio,g_ivfho,g_month,h_hidp,h_pno,h_ivfio,h_ivfho,h_month,i_hidp,i_pno,i_ivfio,i_ivfho,i_month,genetics,epigenetics,xwdat_dv,scend_dv,school_dv,bornuk_dv,generation,evercoh_dv,evermar_dv,anychild_dv,ethn_dv_source,prob91e,prob91w,prob91s,prob99w,prob99s,prob01ni,prob09ni,prob09e,prob09w,prob09s,bb_mortbh_tw,bc_mortbh_tw,bd_mortbh_tw,be_mortbh_tw,bf_mortbh_tw,bg_mortbh_tw,bh_mortbh_tw,bi_mortbh_tw,bj_mortbh_tw,bk_mortbh_tw,bl_mortbh_tw,bm_mortbh_tw,bn_mortbh_tw,bo_mortbh_tw,bp_mortbh_tw,bq_mortbh_tw,br_mortbh_tw,b_mortbh_tw,c_mortbh_tw,d_mortbh_tw,e_mortbh_tw,f_mortbh_tw,g_mortbh_tw,h_mortbh_tw,i_mortbh_tw,b_mortus_tw,c_mortus_tw,d_mortus_tw,e_mortus_tw,f_mortus_tw,g_mortus_tw,h_mortus_tw,psnenub_xd
0,22445,f,6,3.4,25,278664010,3,3,3,4,2,1,6,1,11,2,29,1984,26,6,2014,2,1,2,1,2,2,2,1,0,0,0,0,0,0,1,0,2,2,1,2,1,0,0,0,0,0,0,0,0,2,5,2,3,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,2,1,18,16,57,19,7,12,0,0,2572.590088,0,2572.590088,0,2012.00,0,0.0,0,0.000000,90,72.0,2,29,26,6,2014,1984,2,1,6,1,1,0,0,0,1,0.0,0.0,0.0,0.0,0.000000,2012.000000,1,7,1,1,0,6,3,5,0,0,1,1,17,3,2,0,0,2,2,2,272012925,1,0,0,0,0,1,1,0,0,-8,0,3,31,62.12,32.59,1,0,0,0,0,0.00,0.0,0.00,0.0,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,14,4,14,4,-9,-9,-9,-9

To speed things up, we'll only use observations from wave c (the largest wave, with 40,509 obs)

In [ ]:
# Drop all rows with non-F wave observations
df = df_full[df_full['wave'] == 'f']
print("Shape after dropping non-F wave observations:", df.shape)
df.head(20)

Great! Now we need to deal with missing and negative values. 

First, let's see which columns have the most missing and negative values.

Then, we can drop those columns.

Finally, we'll drop any remaining rows with missing or negative values.

In [3]:
# Calculate % missing and % negative for each column
missing_pct = df.isnull().mean() 
negative_pct = (df.select_dtypes(include=['number']) < 0).mean() * 100

# Combine into a summary DataFrame and sort by the sum of % missing and % negative (descending)
summary = pd.DataFrame({
    '% Missing': missing_pct,
    '% Negative': negative_pct
}).fillna(0)

summary['% Total'] = summary['% Missing'] + summary['% Negative']
summary = summary.sort_values(by='% Total', ascending=False)

summary.head(50)

NameError: name 'df' is not defined

In [ ]:
# Delete the top 25 columns with the highest % missing + % negative
cols_to_drop = summary.head(25).index
df = df.drop(columns=cols_to_drop)

print("Shape after dropping top 25 columns with highest % missing + % negative:", df.shape)
df.head()

Shape after dropping top 25 columns with highest % missing + % negative: (35472, 318)


,pidp,wave,wave_num,nbrsnci_dv,scghq1_dv,hidp,pno,hhorig,memorig,psu,strata,sampst,month,ivfio,ioutcome,sex,dvage,birthy,istrtdatd,istrtdatm,istrtdaty,lkmove,jbstat,racel_dv,health,aidxhh,j2has,bensta2,bensta3,bensta4,bensta5,bensta6,bensta7,bensta96,finnow,finfut,vote1,vote6,mobuse,nch14resp,nch415resp,nchresp,nnatch,nadoptch,nchunder16,nch5to15,nch10to15,sclfsat1,sclfsat2,sclfsat7,sclfsato,marstat,employ,hgbiom,hgbiof,hgpart,respf16,respm16,intdatd_if,intdatm_if,intdaty_if,doby_if,age_if,pn1pno,pn2pno,pns1pno,pns2pno,hhsize,jbhas,istrtdathh,istrtdatmm,istrtdatss,ienddathh,ienddatmm,ienddatss,j2pay_if,fimngrs_tc,fimngrs_dv,fimnlabgrs_tc,fimnlabgrs_dv,fimnlabnet_tc,fimnlabnet_dv,fiyrinvinc_tc,fiyrinvinc_dv,fibenothr_tc,fibenothr_dv,j2pay_dv,j2paynet_dv,sex_dv,age_dv,intdatd_dv,intdatm_dv,intdaty_dv,doby_dv,pensioner_dv,npensioner_dv,marstat_dv,npn_dv,npns_dv,ngrp_dv,nnsib_dv,nnssib_dv,ethn_dv,fimnmisc_dv,fimnprben_dv,fimninvnet_dv,fimnpen_dv,fimnsben_dv,fimnnet_dv,country,gor_dv,urban_dv,hhresp_dv,xtra5min_dv,agegr5_dv,agegr10_dv,agegr13_dv,livesp_dv,cohab_dv,single_dv,mastat_dv,hhtype_dv,buno_dv,depchl_dv,nchild_dv,respm16_dv,respf16_dv,rach16_dv,hrpid,hrpno,ppno,sppno,fnpno,fnspno,mnpno,mnspno,grfpno,grmpno,qfhighfl_dv,hiqual_dv,jbiindb_dv,sf12pcs_dv,sf12mcs_dv,scflag_dv,paygu_if,paynu_if,seearngrs_if,fiyrinvinc_if,fibenothr_if,fimnlabgrs_if,fimngrs_if,ind5mus_xw,ivfho,intdated,intdatem,intdatey,ivh1,ivh2,ivh3,ivh4,ivh5,ivh6,ivh7,ivh8,ivh9,ivh10,ivh11,ivh12,ivh13,ivh14,ivh15,ivh16,hsbeds,hsrooms,hsownd,fuelhave1,fuelhave2,fuelhave3,fuelhave4,fuelhave96,fuelduel,heatch,xphsdct,xphsdba,cduse1,cduse2,cduse5,cduse6,cduse7,cduse8,cduse9,cduse12,cduse13,cduse96,pcnet,xpfood1_g3,xpfdout_g3,xpaltob_g3,ncars,hhintlang,n10to15,fihhmngrs_dv,fihhmngrs_tc,fihhmnlabgrs_dv,fihhmnlabgrs_tc,ctband_if,fihhmnnet1_dv,fihhmnlabnet_dv,fihhmnmisc_dv,fihhmnprben_dv,fihhmninv_dv,fihhmnpen_dv,fihhmnsben_dv,houscost1_dv,houscost2_dv,fihhmngrs1_dv,ctband_dv,ncouple_dv,nonepar_dv,nkids_dv,nch02_dv,nch34_dv,nch511_dv,nch1215_dv,npens_dv,nemp_dv,nue_dv,nwage_dv,nchoecd_dv,nadoecd_dv,ieqmoecd_dv,tenure_dv,fihhnegsei_if,fihhmngrs_if,issue_num,aintlen,outcome,ivtnc,w6osmflag,dcsedfl_dv,lwenum_dv,fwenum_dv,lwintvd_dv,fwintvd_dv,d_hidp,d_pno,d_ivfio,d_ivfho,e_hidp,e_pno,e_ivfio,e_ivfho,e_month,f_hidp,f_pno,f_ivfio,f_ivfho,f_month,g_hidp,g_pno,g_ivfio,g_ivfho,g_month,h_hidp,h_ivfio,h_ivfho,genetics,epigenetics,xwdat_dv,school_dv,bornuk_dv,evercoh_dv,evermar_dv,anychild_dv,ethn_dv_source,prob91e,prob91w,prob91s,prob99w,prob99s,prob01ni,prob09ni,prob09e,prob09w,prob09s,bb_mortbh_tw,bc_mortbh_tw,bd_mortbh_tw,be_mortbh_tw,bf_mortbh_tw,bg_mortbh_tw,bh_mortbh_tw,bi_mortbh_tw,bj_mortbh_tw,bk_mortbh_tw,bl_mortbh_tw,bm_mortbh_tw,bn_mortbh_tw,bo_mortbh_tw,bp_mortbh_tw,bq_mortbh_tw,br_mortbh_tw,b_mortbh_tw,c_mortbh_tw,d_mortbh_tw,e_mortbh_tw,f_mortbh_tw,g_mortbh_tw,h_mortbh_tw,i_mortbh_tw,b_mortus_tw,c_mortus_tw,d_mortus_tw,e_mortus_tw,f_mortus_tw,g_mortus_tw,h_mortus_tw,psnenub_xd
0,22445,f,6,3.4,25,278664010,3,3,3,4,2,1,6,1,11,2,29,1984,26,6,2014,2,2,1,2,2,1,0,0,0,0,0,0,1,2,2,1,2,1,0,0,0,0,0,0,0,0,2,5,2,3,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,2,1,18,16,57,19,7,12,0,0,2572.590088,0,2572.590088,0,2012.000000,0,0.000000,0,0.000000,90,72.0,2,29,26,6,2014,1984,2,1,6,1,1,0,0,0,1,0.0,0.0,0.00,0.0,0.000000,2012.000000,1,7,1,1,0,6,3,5,0,0,1,1,17,3,2,0,2,2,2,272012925,1,0,0,0,0,1,1,0,0,0,3,31,62.12,32.59,1,0,0,0,0,0.00,0.0,0.00,0.0,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2,14,4,14,4,280942006,5,1,10,279255608,5,1,10,4,278664010,3,1,14,6,278447092,1,1,10,6,278092814,1,10,0,0,3,3,1,1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.999978,0.999948,0.999920,0.999890,0.999854,0.999813,0.9997

In [ ]:
# Drop rows with any missing values
df = df.dropna()  # Remove rows with any NaN values

print("Shape after dropping rows with missing values:", df.shape)

df.head()

Shape after dropping rows with missing values: (35131, 318)


,pidp,wave,wave_num,nbrsnci_dv,scghq1_dv,hidp,pno,hhorig,memorig,psu,strata,sampst,month,ivfio,ioutcome,sex,dvage,birthy,istrtdatd,istrtdatm,istrtdaty,lkmove,jbstat,racel_dv,health,aidxhh,j2has,bensta2,bensta3,bensta4,bensta5,bensta6,bensta7,bensta96,finnow,finfut,vote1,vote6,mobuse,nch14resp,nch415resp,nchresp,nnatch,nadoptch,nchunder16,nch5to15,nch10to15,sclfsat1,sclfsat2,sclfsat7,sclfsato,marstat,employ,hgbiom,hgbiof,hgpart,respf16,respm16,intdatd_if,intdatm_if,intdaty_if,doby_if,age_if,pn1pno,pn2pno,pns1pno,pns2pno,hhsize,jbhas,istrtdathh,istrtdatmm,istrtdatss,ienddathh,ienddatmm,ienddatss,j2pay_if,fimngrs_tc,fimngrs_dv,fimnlabgrs_tc,fimnlabgrs_dv,fimnlabnet_tc,fimnlabnet_dv,fiyrinvinc_tc,fiyrinvinc_dv,fibenothr_tc,fibenothr_dv,j2pay_dv,j2paynet_dv,sex_dv,age_dv,intdatd_dv,intdatm_dv,intdaty_dv,doby_dv,pensioner_dv,npensioner_dv,marstat_dv,npn_dv,npns_dv,ngrp_dv,nnsib_dv,nnssib_dv,ethn_dv,fimnmisc_dv,fimnprben_dv,fimninvnet_dv,fimnpen_dv,fimnsben_dv,fimnnet_dv,country,gor_dv,urban_dv,hhresp_dv,xtra5min_dv,agegr5_dv,agegr10_dv,agegr13_dv,livesp_dv,cohab_dv,single_dv,mastat_dv,hhtype_dv,buno_dv,depchl_dv,nchild_dv,respm16_dv,respf16_dv,rach16_dv,hrpid,hrpno,ppno,sppno,fnpno,fnspno,mnpno,mnspno,grfpno,grmpno,qfhighfl_dv,hiqual_dv,jbiindb_dv,sf12pcs_dv,sf12mcs_dv,scflag_dv,paygu_if,paynu_if,seearngrs_if,fiyrinvinc_if,fibenothr_if,fimnlabgrs_if,fimngrs_if,ind5mus_xw,ivfho,intdated,intdatem,intdatey,ivh1,ivh2,ivh3,ivh4,ivh5,ivh6,ivh7,ivh8,ivh9,ivh10,ivh11,ivh12,ivh13,ivh14,ivh15,ivh16,hsbeds,hsrooms,hsownd,fuelhave1,fuelhave2,fuelhave3,fuelhave4,fuelhave96,fuelduel,heatch,xphsdct,xphsdba,cduse1,cduse2,cduse5,cduse6,cduse7,cduse8,cduse9,cduse12,cduse13,cduse96,pcnet,xpfood1_g3,xpfdout_g3,xpaltob_g3,ncars,hhintlang,n10to15,fihhmngrs_dv,fihhmngrs_tc,fihhmnlabgrs_dv,fihhmnlabgrs_tc,ctband_if,fihhmnnet1_dv,fihhmnlabnet_dv,fihhmnmisc_dv,fihhmnprben_dv,fihhmninv_dv,fihhmnpen_dv,fihhmnsben_dv,houscost1_dv,houscost2_dv,fihhmngrs1_dv,ctband_dv,ncouple_dv,nonepar_dv,nkids_dv,nch02_dv,nch34_dv,nch511_dv,nch1215_dv,npens_dv,nemp_dv,nue_dv,nwage_dv,nchoecd_dv,nadoecd_dv,ieqmoecd_dv,tenure_dv,fihhnegsei_if,fihhmngrs_if,issue_num,aintlen,outcome,ivtnc,w6osmflag,dcsedfl_dv,lwenum_dv,fwenum_dv,lwintvd_dv,fwintvd_dv,d_hidp,d_pno,d_ivfio,d_ivfho,e_hidp,e_pno,e_ivfio,e_ivfho,e_month,f_hidp,f_pno,f_ivfio,f_ivfho,f_month,g_hidp,g_pno,g_ivfio,g_ivfho,g_month,h_hidp,h_ivfio,h_ivfho,genetics,epigenetics,xwdat_dv,school_dv,bornuk_dv,evercoh_dv,evermar_dv,anychild_dv,ethn_dv_source,prob91e,prob91w,prob91s,prob99w,prob99s,prob01ni,prob09ni,prob09e,prob09w,prob09s,bb_mortbh_tw,bc_mortbh_tw,bd_mortbh_tw,be_mortbh_tw,bf_mortbh_tw,bg_mortbh_tw,bh_mortbh_tw,bi_mortbh_tw,bj_mortbh_tw,bk_mortbh_tw,bl_mortbh_tw,bm_mortbh_tw,bn_mortbh_tw,bo_mortbh_tw,bp_mortbh_tw,bq_mortbh_tw,br_mortbh_tw,b_mortbh_tw,c_mortbh_tw,d_mortbh_tw,e_mortbh_tw,f_mortbh_tw,g_mortbh_tw,h_mortbh_tw,i_mortbh_tw,b_mortus_tw,c_mortus_tw,d_mortus_tw,e_mortus_tw,f_mortus_tw,g_mortus_tw,h_mortus_tw,psnenub_xd
3,29925,f,6,4.1,11,620547610,1,3,3,6,2,1,8,1,11,2,37,1977,29,9,2014,1,1,1,1,2,2,0,0,1,0,0,0,0,4,1,2,2,1,2,1,2,2,0,2,0,0,3,2,5,4,4,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,1,10,0,47,10,48,3,0,0,2175.620117,0,13.82,0,13.820000,0,0.000000,0,2161.800049,0,0.0,2,37,29,9,2014,1977,2,0,5,0,0,0,0,0,1,0.0,320.0,0.00,0.0,1841.800049,2175.620117,1,7,1,1,0,8,4,7,0,0,1,4,5,1,2,2,1,2,1,29925,1,0,0,0,0,0,0,0,0,0,1,30,56.59,35.67,1,0,0,1,0,0.04,1.0,0.05,0.0,10,29,9,2014,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,4,1,1,0,0,0,1,1,2,1,1,1,1,1,0,1,1,1,1,0,1,350,30,0,1,0,0,2175.620117,0,13.820000,0,3,2175.620117,13.820000,0.0,320.0,0.000000,0.0,1841.800049,1451.0,1451.000000,2175.620117,2,0,1,2,0,2,0,0,0,1,0,1,2,1,1.6,7,0,0.0451,1,10.0,110,3,0,2,14,4,14,4,622866406,3,2,11,621384688,1,50,61,9,620547610,1,1,10,8,620316412,1,1,10,8,619935614,1,10,0,0,3,3,1,1,1,1,2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.999955,0.999895,0.999839,0.999777,0.999704,0.999622,0.999538,0.999470,0.999369,0.999289,0.999208,0.999120,0.998890,0.998

In [ ]:
# Drop rows with any negative values
numeric_cols = df.select_dtypes(include='number').columns
df = df[(df[numeric_cols] >= 0).all(axis=1)]
print("Shape after dropping rows with negative values:", df.shape)
df.head()

Shape after dropping rows with negative values: (30393, 318)


,pidp,wave,wave_num,nbrsnci_dv,scghq1_dv,hidp,pno,hhorig,memorig,psu,strata,sampst,month,ivfio,ioutcome,sex,dvage,birthy,istrtdatd,istrtdatm,istrtdaty,lkmove,jbstat,racel_dv,health,aidxhh,j2has,bensta2,bensta3,bensta4,bensta5,bensta6,bensta7,bensta96,finnow,finfut,vote1,vote6,mobuse,nch14resp,nch415resp,nchresp,nnatch,nadoptch,nchunder16,nch5to15,nch10to15,sclfsat1,sclfsat2,sclfsat7,sclfsato,marstat,employ,hgbiom,hgbiof,hgpart,respf16,respm16,intdatd_if,intdatm_if,intdaty_if,doby_if,age_if,pn1pno,pn2pno,pns1pno,pns2pno,hhsize,jbhas,istrtdathh,istrtdatmm,istrtdatss,ienddathh,ienddatmm,ienddatss,j2pay_if,fimngrs_tc,fimngrs_dv,fimnlabgrs_tc,fimnlabgrs_dv,fimnlabnet_tc,fimnlabnet_dv,fiyrinvinc_tc,fiyrinvinc_dv,fibenothr_tc,fibenothr_dv,j2pay_dv,j2paynet_dv,sex_dv,age_dv,intdatd_dv,intdatm_dv,intdaty_dv,doby_dv,pensioner_dv,npensioner_dv,marstat_dv,npn_dv,npns_dv,ngrp_dv,nnsib_dv,nnssib_dv,ethn_dv,fimnmisc_dv,fimnprben_dv,fimninvnet_dv,fimnpen_dv,fimnsben_dv,fimnnet_dv,country,gor_dv,urban_dv,hhresp_dv,xtra5min_dv,agegr5_dv,agegr10_dv,agegr13_dv,livesp_dv,cohab_dv,single_dv,mastat_dv,hhtype_dv,buno_dv,depchl_dv,nchild_dv,respm16_dv,respf16_dv,rach16_dv,hrpid,hrpno,ppno,sppno,fnpno,fnspno,mnpno,mnspno,grfpno,grmpno,qfhighfl_dv,hiqual_dv,jbiindb_dv,sf12pcs_dv,sf12mcs_dv,scflag_dv,paygu_if,paynu_if,seearngrs_if,fiyrinvinc_if,fibenothr_if,fimnlabgrs_if,fimngrs_if,ind5mus_xw,ivfho,intdated,intdatem,intdatey,ivh1,ivh2,ivh3,ivh4,ivh5,ivh6,ivh7,ivh8,ivh9,ivh10,ivh11,ivh12,ivh13,ivh14,ivh15,ivh16,hsbeds,hsrooms,hsownd,fuelhave1,fuelhave2,fuelhave3,fuelhave4,fuelhave96,fuelduel,heatch,xphsdct,xphsdba,cduse1,cduse2,cduse5,cduse6,cduse7,cduse8,cduse9,cduse12,cduse13,cduse96,pcnet,xpfood1_g3,xpfdout_g3,xpaltob_g3,ncars,hhintlang,n10to15,fihhmngrs_dv,fihhmngrs_tc,fihhmnlabgrs_dv,fihhmnlabgrs_tc,ctband_if,fihhmnnet1_dv,fihhmnlabnet_dv,fihhmnmisc_dv,fihhmnprben_dv,fihhmninv_dv,fihhmnpen_dv,fihhmnsben_dv,houscost1_dv,houscost2_dv,fihhmngrs1_dv,ctband_dv,ncouple_dv,nonepar_dv,nkids_dv,nch02_dv,nch34_dv,nch511_dv,nch1215_dv,npens_dv,nemp_dv,nue_dv,nwage_dv,nchoecd_dv,nadoecd_dv,ieqmoecd_dv,tenure_dv,fihhnegsei_if,fihhmngrs_if,issue_num,aintlen,outcome,ivtnc,w6osmflag,dcsedfl_dv,lwenum_dv,fwenum_dv,lwintvd_dv,fwintvd_dv,d_hidp,d_pno,d_ivfio,d_ivfho,e_hidp,e_pno,e_ivfio,e_ivfho,e_month,f_hidp,f_pno,f_ivfio,f_ivfho,f_month,g_hidp,g_pno,g_ivfio,g_ivfho,g_month,h_hidp,h_ivfio,h_ivfho,genetics,epigenetics,xwdat_dv,school_dv,bornuk_dv,evercoh_dv,evermar_dv,anychild_dv,ethn_dv_source,prob91e,prob91w,prob91s,prob99w,prob99s,prob01ni,prob09ni,prob09e,prob09w,prob09s,bb_mortbh_tw,bc_mortbh_tw,bd_mortbh_tw,be_mortbh_tw,bf_mortbh_tw,bg_mortbh_tw,bh_mortbh_tw,bi_mortbh_tw,bj_mortbh_tw,bk_mortbh_tw,bl_mortbh_tw,bm_mortbh_tw,bn_mortbh_tw,bo_mortbh_tw,bp_mortbh_tw,bq_mortbh_tw,br_mortbh_tw,b_mortbh_tw,c_mortbh_tw,d_mortbh_tw,e_mortbh_tw,f_mortbh_tw,g_mortbh_tw,h_mortbh_tw,i_mortbh_tw,b_mortus_tw,c_mortus_tw,d_mortus_tw,e_mortus_tw,f_mortus_tw,g_mortus_tw,h_mortus_tw,psnenub_xd
3,29925,f,6,4.1,11,620547610,1,3,3,6,2,1,8,1,11,2,37,1977,29,9,2014,1,1,1,1,2,2,0,0,1,0,0,0,0,4,1,2,2,1,2,1,2,2,0,2,0,0,3,2,5,4,4,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,1,10,0,47,10,48,3,0,0,2175.620117,0,13.82,0,13.820000,0,0.000000,0,2161.800049,0,0.0,2,37,29,9,2014,1977,2,0,5,0,0,0,0,0,1,0.0,320.0,0.00,0.0,1841.800049,2175.620117,1,7,1,1,0,8,4,7,0,0,1,4,5,1,2,2,1,2,1,29925,1,0,0,0,0,0,0,0,0,0,1,30,56.59,35.67,1,0,0,1,0,0.04,1.0,0.05,0.0,10,29,9,2014,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,4,1,1,0,0,0,1,1,2,1,1,1,1,1,0,1,1,1,1,0,1,350,30,0,1,0,0,2175.620117,0,13.820000,0,3,2175.620117,13.820000,0.0,320.0,0.000000,0.0,1841.800049,1451.0,1451.000000,2175.620117,2,0,1,2,0,2,0,0,0,1,0,1,2,1,1.6,7,0,0.0451,1,10.0,110,3,0,2,14,4,14,4,622866406,3,2,11,621384688,1,50,61,9,620547610,1,1,10,8,620316412,1,1,10,8,619935614,1,10,0,0,3,3,1,1,1,1,2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.999955,0.999895,0.999839,0.999777,0.999704,0.999622,0.999538,0.999470,0.999369,0.999289,0.999208,0.999120,0.998890,0.998

Great! Now we need to take out scghq1_dv (the outcome variable) and drop identifiers (pidp wave wave_num hidp)

In [ ]:
# Separate outcome (scghq1_dv) and predictors (X)

Y = df['scghq1_dv'].astype(float)  # Ensure Y is a numeric pandas Series
X = df.drop(columns=['scghq1_dv'])

# Drop identifiers (pidp, wave, wave_num, hidp, pno, hhorig, memorig, psu, strata, sampst, month, ivfio, ioutcome, birthy, hrpid)
identifiers = ['pidp', 'wave', 'wave_num', 'hidp', 'pno', 'hhorig', 'memorig', 'psu', 'strata', 'sampst', 'month', 'ivfio', 'ioutcome', 'birthy', 'hrpid']
X = X.drop(columns=identifiers)

In [ ]:
# Strict rule: discrete only if dtype is object/category or integer dtype, and num_unique <= MAX_UNIQUE_DISCRETE
# Anything with more than MAX_UNIQUE_DISCRETE unique values is automatically continuous.

MAX_UNIQUE_DISCRETE = 30

label_map = dict(zip(meta.column_names, meta.column_labels)) if hasattr(meta, 'column_names') else {}
rows = []

for col in X.columns:
    dtype = X[col].dtype
    col_vals = X[col].dropna()
    num_unique = int(col_vals.nunique())

    # Automatic continuous if too many uniques
    if num_unique > MAX_UNIQUE_DISCRETE:
        will_discrete = False
        reason = f'{num_unique} unique > {MAX_UNIQUE_DISCRETE} -> continuous'
    else:
        is_string = dtype == 'object' or str(dtype).startswith('category')
        is_integer = pd.api.types.is_integer_dtype(X[col].dtype)

        will_discrete = bool(is_string or is_integer)
        if is_string:
            reason = 'string/category dtype -> discrete'
        elif is_integer:
            reason = 'integer dtype -> discrete'
        else:
            reason = 'float/numeric dtype -> continuous'

    suggested_action = 'one-hot encode (discrete)' if will_discrete else 'treat as continuous'
    sample_vals = list(pd.Series(col_vals.unique()).sort_values()[:6]) if num_unique > 0 else []

    rows.append({
        'Column': col,
        'Label': label_map.get(col, ''),
        'DataType': str(dtype),
        'NumUnique': num_unique,
        'IsString': dtype == 'object' or str(dtype).startswith('category'),
        'IsInteger': pd.api.types.is_integer_dtype(X[col].dtype),
        'WillBeDiscrete': will_discrete,
        'DecisionReason': reason,
        'SuggestedAction': suggested_action
    })

variable_summary = pd.DataFrame(rows)
# Order discrete first for visibility
variable_summary = variable_summary.sort_values(by=['WillBeDiscrete', 'NumUnique'], ascending=[False, True]).reset_index(drop=True)

print(f"Total variables: {len(variable_summary)}")
print(f"Discrete by rule: {int(variable_summary['WillBeDiscrete'].sum())}")
print(f"MAX_UNIQUE_DISCRETE = {MAX_UNIQUE_DISCRETE}")

pd.set_option('display.max_rows', None)
display(variable_summary)


Total variables: 302
Discrete by rule: 204
MAX_UNIQUE_DISCRETE = 30


,Column,Label,DataType,NumUnique,IsString,IsInteger,WillBeDiscrete,DecisionReason,SuggestedAction
0,intdatd_if,"Interview date: Day, imputation flag",int64,1,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
1,intdatm_if,"Interview date: Month, imputation flag",int64,1,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
2,intdaty_if,"Interview date: Year, imputation flag",int64,1,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
3,doby_if,DOB Year imputation flag,int64,1,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
4,age_if,Imputation flag for age_dv,int64,1,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
5,f_ivfio,individual interview outcome,int64,1,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
6,sex,Sex,int64,2,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
7,lkmove,Prefers to move house,int64,2,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
8,health,Long-standing illness or disability,int64,2,False,True,True,integer dtype -> discrete,one-hot encode (discrete)
9,aidxhh,Non-residents cared for,int64,2,False,True,True,integer dtype -> discrete,one-hot encode (discrete)


Great! Now we need to scale the continuous variables.

In [ ]:
# Scale continuous variables to mean 0, std 1
from sklearn.preprocessing import StandardScaler
import numpy as np

# Identify continuous columns using the variable_summary decisions
continuous_cols = variable_summary.loc[~variable_summary['WillBeDiscrete'], 'Column'].tolist()

# Only keep columns that still exist in X (defensive)
continuous_cols = [c for c in continuous_cols if c in X.columns]

print(f'Found {len(continuous_cols)} continuous column(s) to scale')

# Keep a copy of the unscaled X in case we need it later
X_unscaled = X.copy()

scaler = None
if len(continuous_cols) > 0:
    scaler = StandardScaler()
    # Convert to float (safe) and scale in-place on a copy
    X_scaled = X.copy()
    try:
        X_scaled[continuous_cols] = scaler.fit_transform(X_scaled[continuous_cols].astype(float))
    except Exception as e:
        # Fall back to scaling each column separately if there are issues with mixed dtypes
        print('Warning: bulk scaling failed, falling back to per-column scaling. Error:', e)
        for col in continuous_cols:
            try:
                vals = X_scaled[col].astype(float).values.reshape(-1, 1)
                X_scaled[col] = scaler.fit_transform(vals).ravel()
            except Exception as e2:
                print(f'  Could not scale column {col}:', e2)
    
    # Replace X with scaled version
    X = X_scaled
else:
    print('No continuous columns to scale; X left unchanged')

print('Shape of X after scaling:', X.shape)

# Quick sanity checks
if scaler is not None and len(continuous_cols) > 0:
    # show means and stds (approx) for a few columns
    sample_check = continuous_cols[:6]
    means = X[sample_check].mean().round(6)
    stds = X[sample_check].std().round(6)
    print('Sample scaled means (should be near 0):')
    print(means.to_dict())
    print('Sample scaled stds (should be near 1):')
    print(stds.to_dict())

# Display head for quick verification
pd.set_option('display.max_columns', None)
display(X.head())

Found 98 continuous column(s) to scale
Shape of X after scaling: (30393, 302)
Sample scaled means (should be near 0):
{'b_mortus_tw': 0.0, 'prob01ni': 0.0, 'c_mortus_tw': 0.0, 'prob91e': 0.0, 'prob91w': 0.0, 'prob91s': 0.0}
Sample scaled stds (should be near 1):
{'b_mortus_tw': 0.0, 'prob01ni': 1.000016, 'c_mortus_tw': 1.000016, 'prob91e': 1.000016, 'prob91w': 1.000016, 'prob91s': 1.000016}


,nbrsnci_dv,sex,dvage,istrtdatd,istrtdatm,istrtdaty,lkmove,jbstat,racel_dv,health,aidxhh,j2has,bensta2,bensta3,bensta4,bensta5,bensta6,bensta7,bensta96,finnow,finfut,vote1,vote6,mobuse,nch14resp,nch415resp,nchresp,nnatch,nadoptch,nchunder16,nch5to15,nch10to15,sclfsat1,sclfsat2,sclfsat7,sclfsato,marstat,employ,hgbiom,hgbiof,hgpart,respf16,respm16,intdatd_if,intdatm_if,intdaty_if,doby_if,age_if,pn1pno,pn2pno,pns1pno,pns2pno,hhsize,jbhas,istrtdathh,istrtdatmm,istrtdatss,ienddathh,ienddatmm,ienddatss,j2pay_if,fimngrs_tc,fimngrs_dv,fimnlabgrs_tc,fimnlabgrs_dv,fimnlabnet_tc,fimnlabnet_dv,fiyrinvinc_tc,fiyrinvinc_dv,fibenothr_tc,fibenothr_dv,j2pay_dv,j2paynet_dv,sex_dv,age_dv,intdatd_dv,intdatm_dv,intdaty_dv,doby_dv,pensioner_dv,npensioner_dv,marstat_dv,npn_dv,npns_dv,ngrp_dv,nnsib_dv,nnssib_dv,ethn_dv,fimnmisc_dv,fimnprben_dv,fimninvnet_dv,fimnpen_dv,fimnsben_dv,fimnnet_dv,country,gor_dv,urban_dv,hhresp_dv,xtra5min_dv,agegr5_dv,agegr10_dv,agegr13_dv,livesp_dv,cohab_dv,single_dv,mastat_dv,hhtype_dv,buno_dv,depchl_dv,nchild_dv,respm16_dv,respf16_dv,rach16_dv,hrpno,ppno,sppno,fnpno,fnspno,mnpno,mnspno,grfpno,grmpno,qfhighfl_dv,hiqual_dv,jbiindb_dv,sf12pcs_dv,sf12mcs_dv,scflag_dv,paygu_if,paynu_if,seearngrs_if,fiyrinvinc_if,fibenothr_if,fimnlabgrs_if,fimngrs_if,ind5mus_xw,ivfho,intdated,intdatem,intdatey,ivh1,ivh2,ivh3,ivh4,ivh5,ivh6,ivh7,ivh8,ivh9,ivh10,ivh11,ivh12,ivh13,ivh14,ivh15,ivh16,hsbeds,hsrooms,hsownd,fuelhave1,fuelhave2,fuelhave3,fuelhave4,fuelhave96,fuelduel,heatch,xphsdct,xphsdba,cduse1,cduse2,cduse5,cduse6,cduse7,cduse8,cduse9,cduse12,cduse13,cduse96,pcnet,xpfood1_g3,xpfdout_g3,xpaltob_g3,ncars,hhintlang,n10to15,fihhmngrs_dv,fihhmngrs_tc,fihhmnlabgrs_dv,fihhmnlabgrs_tc,ctband_if,fihhmnnet1_dv,fihhmnlabnet_dv,fihhmnmisc_dv,fihhmnprben_dv,fihhmninv_dv,fihhmnpen_dv,fihhmnsben_dv,houscost1_dv,houscost2_dv,fihhmngrs1_dv,ctband_dv,ncouple_dv,nonepar_dv,nkids_dv,nch02_dv,nch34_dv,nch511_dv,nch1215_dv,npens_dv,nemp_dv,nue_dv,nwage_dv,nchoecd_dv,nadoecd_dv,ieqmoecd_dv,tenure_dv,fihhnegsei_if,fihhmngrs_if,issue_num,aintlen,outcome,ivtnc,w6osmflag,dcsedfl_dv,lwenum_dv,fwenum_dv,lwintvd_dv,fwintvd_dv,d_hidp,d_pno,d_ivfio,d_ivfho,e_hidp,e_pno,e_ivfio,e_ivfho,e_month,f_hidp,f_pno,f_ivfio,f_ivfho,f_month,g_hidp,g_pno,g_ivfio,g_ivfho,g_month,h_hidp,h_ivfio,h_ivfho,genetics,epigenetics,xwdat_dv,school_dv,bornuk_dv,evercoh_dv,evermar_dv,anychild_dv,ethn_dv_source,prob91e,prob91w,prob91s,prob99w,prob99s,prob01ni,prob09ni,prob09e,prob09w,prob09s,bb_mortbh_tw,bc_mortbh_tw,bd_mortbh_tw,be_mortbh_tw,bf_mortbh_tw,bg_mortbh_tw,bh_mortbh_tw,bi_mortbh_tw,bj_mortbh_tw,bk_mortbh_tw,bl_mortbh_tw,bm_mortbh_tw,bn_mortbh_tw,bo_mortbh_tw,bp_mortbh_tw,bq_mortbh_tw,br_mortbh_tw,b_mortbh_tw,c_mortbh_tw,d_mortbh_tw,e_mortbh_tw,f_mortbh_tw,g_mortbh_tw,h_mortbh_tw,i_mortbh_tw,b_mortus_tw,c_mortus_tw,d_mortus_tw,e_mortus_tw,f_mortus_tw,g_mortus_tw,h_mortus_tw,psnenub_xd
3,0.639878,2,-0.672515,1.692497,9,2014,1,1,1,1,2,2,0,0,1,0,0,0,0,4,1,2,2,1,2,1,2,2,0,2,0,0,3,2,5,4,4,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,1,10,-1.669109,1.010297,10,1.060204,-1.516673,0,0,0.211878,0,-0.741780,0,-0.808718,0,-0.145866,0,1.954234,-0.1074,-0.115447,2,-0.672223,1.692497,9,2014,0.674773,2,0,5,0,0,0,0,0,1,-0.112401,3.941976,-0.202415,-0.297459,3.418842,0.127134,1,7,1,1,0,8,4,7,0,0,1,4,5,1,2,2,1,2,1,1,0,0,0,0,0,0,0,0,0,1,1.260867,0.628624,-1.485992,1,0,0,1,0,-0.154913,3.520283,-0.256349,-0.11085,10,1.675516,9,2014,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,4,1,1,0,0,0,1,1,2,1,1,1,1,1,0,1,1,1,1,0,1,0.143739,-0.541800,-0.664290,1,0,0,-0.640248,0,-0.958280,0,3,-0.187982,-0.329248,-0.17949,2.567482,-0.282252,-0.366117,1.888175,2.196130,3.374203,-0.170821,2,0,1,2,0,2,0,0,0,1,0,1,2,1,-0.367597,7,0,-0.513697,1,-0.255135,110,-0.441872,0,2,14,4,14,4,-0.272483,3,2,11,-0.273698,1,50,61,9,-0.274659,1,1,10,8,-0.274879,1,1,10,8,-0.275217,1,10,0,0,3,3,1,1,1,1,2,-1.361249,-0.255976,-0.295974,-0.258665,-0.294789,-0.25275,-0.17737,-1.248358,-0.183014,-0.228409,0.007908,0.007190,0.006539,0.005842,0.004955,0.003970,0.00

Great, now we need to encode categorical variables using one-hot encoding

In [ ]:
# Encode categorical variables using one-hot encoding, then show shape and head

# Identify categorical (discrete) columns from variable_summary
cat_cols = variable_summary.loc[variable_summary['WillBeDiscrete'], 'Column'].tolist()

# Defensive: keep only those present in X
cat_cols = [c for c in cat_cols if c in X.columns]
print(f'Found {len(cat_cols)} categorical column(s) to encode')

# If there are categorical columns, create dummies and merge with the rest of X
if len(cat_cols) > 0:
    # Convert to string to ensure stable dummy names (preserve distinct categories)
    cat_df = X[cat_cols].astype(str).apply(lambda s: s.str.replace(' ', '_'))
    # Create dummies; drop_first avoids creating a full-rank encoding
    dummies = pd.get_dummies(cat_df, prefix=cat_cols, prefix_sep='_', drop_first=True, dummy_na=False)
    # Build new X: drop original categorical columns, concat dummies
    X_encoded = X.drop(columns=cat_cols).copy()
    # Ensure no column name collisions (rename if necessary)
    overlap = set(X_encoded.columns).intersection(dummies.columns)
    if overlap:
        # Rare: if a dummy name collides with existing column, prefix dummy names with 'dum_'
        dummies = dummies.rename(columns={c: f'dum_{c}' for c in dummies.columns})
    X_encoded = pd.concat([X_encoded, dummies], axis=1)
else:
    print('No categorical columns selected for encoding')
    X_encoded = X.copy()

# Replace X with encoded version for downstream modeling
X = X_encoded

print('Shape of X after encoding:', X.shape)
pd.set_option('display.max_columns', None)
display(X.head())

Found 204 categorical column(s) to encode
Shape of X after encoding: (30393, 1255)
Shape of X after encoding: (30393, 1255)


,nbrsnci_dv,dvage,istrtdatd,istrtdatmm,istrtdatss,ienddatmm,ienddatss,fimngrs_dv,fimnlabgrs_dv,fimnlabnet_dv,fiyrinvinc_dv,fibenothr_dv,j2pay_dv,j2paynet_dv,age_dv,intdatd_dv,doby_dv,fimnmisc_dv,fimnprben_dv,fimninvnet_dv,fimnpen_dv,fimnsben_dv,fimnnet_dv,jbiindb_dv,sf12pcs_dv,sf12mcs_dv,fibenothr_if,fimnlabgrs_if,fimngrs_if,ind5mus_xw,intdated,xpfood1_g3,xpfdout_g3,xpaltob_g3,fihhmngrs_dv,fihhmnlabgrs_dv,fihhmnnet1_dv,fihhmnlabnet_dv,fihhmnmisc_dv,fihhmnprben_dv,fihhmninv_dv,fihhmnpen_dv,fihhmnsben_dv,houscost1_dv,houscost2_dv,fihhmngrs1_dv,ieqmoecd_dv,fihhmngrs_if,aintlen,ivtnc,d_hidp,e_hidp,f_hidp,g_hidp,h_hidp,prob91e,prob91w,prob91s,prob99w,prob99s,prob01ni,prob09ni,prob09e,prob09w,prob09s,bb_mortbh_tw,bc_mortbh_tw,bd_mortbh_tw,be_mortbh_tw,bf_mortbh_tw,bg_mortbh_tw,bh_mortbh_tw,bi_mortbh_tw,bj_mortbh_tw,bk_mortbh_tw,bl_mortbh_tw,bm_mortbh_tw,bn_mortbh_tw,bo_mortbh_tw,bp_mortbh_tw,bq_mortbh_tw,br_mortbh_tw,b_mortbh_tw,c_mortbh_tw,d_mortbh_tw,e_mortbh_tw,f_mortbh_tw,g_mortbh_tw,h_mortbh_tw,i_mortbh_tw,b_mortus_tw,c_mortus_tw,d_mortus_tw,e_mortus_tw,f_mortus_tw,g_mortus_tw,h_mortus_tw,psnenub_xd,sex_2,lkmove_2,health_2,aidxhh_2,j2has_2,bensta2_1,bensta3_1,bensta4_1,bensta5_1,bensta6_1,bensta7_1,bensta96_1,vote1_2,mobuse_2,employ_2,respf16_1,respm16_1,jbhas_2,j2pay_if_1,fimngrs_tc_1,fimnlabgrs_tc_1,fimnlabnet_tc_1,fiyrinvinc_tc_1,fibenothr_tc_1,pensioner_dv_2,urban_dv_2,xtra5min_dv_1,livesp_dv_1,cohab_dv_1,single_dv_1,depchl_dv_2,respm16_dv_2,respf16_dv_2,rach16_dv_2,qfhighfl_dv_1,scflag_dv_1,paygu_if_1,paynu_if_1,seearngrs_if_1,fiyrinvinc_if_1,fihhmngrs_tc_1,fihhmnlabgrs_tc_1,fihhnegsei_if_1,outcome_210,w6osmflag_1,dcsedfl_dv_2,genetics_1,epigenetics_1,xwdat_dv_3,bornuk_dv_2,evercoh_dv_2,evermar_dv_2,anychild_dv_2,istrtdaty_2015,istrtdaty_2016,finfut_2,finfut_3,sex_dv_1,sex_dv_2,intdaty_dv_2015,intdaty_dv_2016,npn_dv_1,npn_dv_2,npns_dv_1,npns_dv_2,ngrp_dv_1,ngrp_dv_2,hhresp_dv_2,hhresp_dv_3,ivfho_11,ivfho_12,intdatey_2015,intdatey_2016,nonepar_dv_1,nonepar_dv_2,f_ivfho_11,f_ivfho_12,school_dv_2,school_dv_3,vote6_2,vote6_3,vote6_4,npensioner_dv_1,npensioner_dv_2,npensioner_dv_3,country_2,country_3,country_4,ivh10_-2,ivh10_-9,ivh10_0,ivh11_-2,ivh11_-9,ivh11_0,ivh12_-2,ivh12_-9,ivh12_0,ivh13_-2,ivh13_-9,ivh13_0,ivh14_-2,ivh14_-9,ivh14_0,ivh15_-2,ivh15_-9,ivh15_0,ivh16_-2,ivh16_-9,ivh16_0,heatch_-2,heatch_1,heatch_2,pcnet_-2,pcnet_1,pcnet_2,ctband_if_1,ctband_if_2,ctband_if_3,nch02_dv_1,nch02_dv_2,nch02_dv_3,nch34_dv_1,nch34_dv_2,nch34_dv_3,npens_dv_1,npens_dv_2,npens_dv_3,ethn_dv_source_2,ethn_dv_source_3,ethn_dv_source_4,finnow_2,finnow_3,finnow_4,finnow_5,nadoptch_1,nadoptch_2,nadoptch_3,nadoptch_4,ivh1_-2,ivh1_-9,ivh1_0,ivh1_1,ivh2_-2,ivh2_-9,ivh2_0,ivh2_1,ivh3_-2,ivh3_-9,ivh3_0,ivh3_1,ivh4_-2,ivh4_-9,ivh4_0,ivh4_1,ivh5_-2,ivh5_-9,ivh5_0,ivh5_1,ivh6_-2,ivh6_-9,ivh6_0,ivh6_1,ivh7_-2,ivh7_-9,ivh7_0,ivh7_1,ivh8_-2,ivh8_-9,ivh8_0,ivh8_1,ivh9_-2,ivh9_-9,ivh9_0,ivh9_1,fuelhave1_-2,fuelhave1_-9,fuelhave1_0,fuelhave1_1,fuelhave2_-2,fuelhave2_-9,fuelhave2_0,fuelhave2_1,fuelhave3_-2,fuelhave3_-9,fuelhave3_0,fuelhave3_1,fuelhave4_-2,fuelhave4_-9,fuelhave4_0,fuelhave4_1,fuelhave96_-2,fuelhave96_-9,fuelhave96_0,fuelhave96_1,fuelduel_-2,fuelduel_-8,fuelduel_1,fuelduel_2,xphsdba_-2,xphsdba_1,xphsdba_2,xphsdba_3,cduse96_-2,cduse96_-9,cduse96_0,cduse96_1,ncouple_dv_1,ncouple_dv_2,ncouple_dv_3,ncouple_dv_4,nch1215_dv_1,nch1215_dv_2,nch1215_dv_3,nch1215_dv_4,issue_num_2,issue_num_3,issue_num_4,issue_num_5,nch10to15_1,nch10to15_2,nch10to15_3,nch10to15_4,nch10to15_5,marstat_dv_2,marstat_dv_3,marstat_dv_4,marstat_dv_5,marstat_dv_6,hiqual_dv_2,hiqual_dv_3,hiqual_dv_4,hiqual_dv_5,hiqual_dv_9,xphsdct_-2,xphsdct_-8,xphsdct_-9,xphsdct_1,xphsdct_2,cduse1_-2,cduse1_-8,cduse1_-9,cduse1_0,cduse1_1,cduse2_-2,cduse2_-8,cduse2_-9,cduse2_0,cduse2_1,cduse5_-2,cduse5_-8,cduse5_-9,cduse5_0,cduse5_1,cduse6_-2,cduse6_-8,cduse6_-9,cduse6_0,cduse6_1,cduse7_-2,cduse7_-8,cduse7_-9,cduse7_0,cduse7_1,cduse8_-2,cduse8_-8,cduse8_-9,cduse8_0,cduse8_1,cduse9_-2,cduse9_-8,c

Lets move on to fitting the lasso model!

In [ ]:
# Build and run Lasso, then map nonzero coefficients back to variable labels (removed BaselineCategory)
from sklearn.linear_model import LassoCV
import numpy as np
import pandas as pd

# Fit Lasso with cross-validation
lasso = LassoCV(cv=5, random_state=0, max_iter=10000)
lasso.fit(X, Y)

# Get nonzero coefficients sorted by absolute value
coefs = pd.Series(lasso.coef_, index=X.columns)
nonzero_coefs = coefs[coefs != 0].reindex(coefs[coefs != 0].abs().sort_values(ascending=False).index)

# Create label mapping from Stata metadata
label_map = dict(zip(meta.column_names, meta.column_labels)) if hasattr(meta, 'column_labels') else {}
original_vars = set(meta.column_names) if hasattr(meta, 'column_names') else set()

def parse_variable(varname):
    """Parse variable to get base variable, category, and labels (no baseline category)."""
    # Find base variable and category
    if varname in original_vars:
        base_var, category = varname, ''
    elif '_' in varname:
        base, cat = varname.rsplit('_', 1)
        if base in original_vars:
            base_var, category = base, cat
        else:
            # Try progressively shorter prefixes for variables with underscores
            parts = varname.split('_')
            base_var, category = varname, ''
            for i in range(len(parts) - 1, 0, -1):
                potential_base = '_'.join(parts[:i])
                if potential_base in original_vars:
                    base_var, category = potential_base, '_'.join(parts[i:])
                    break
    else:
        base_var, category = varname, ''

    # Get variable label
    variable_label = label_map.get(base_var, base_var)

    # Get category label from value labels
    category_label = ''
    if category and hasattr(meta, 'variable_value_labels') and base_var in meta.variable_value_labels:
        value_dict = meta.variable_value_labels[base_var]
        for cat_type in [int, float, str]:
            try:
                cat_key = cat_type(category)
                if cat_key in value_dict:
                    category_label = value_dict[cat_key]
                    break
            except (ValueError, TypeError):
                continue

    return base_var, variable_label, category, category_label

# Build results table (no BaselineCategory)
results_data = []
for var in nonzero_coefs.index:
    base_var, variable_label, category, category_label = parse_variable(var)
    results_data.append({
        'Variable': var,
        'Label': variable_label,
        'Category': category,
        'CategoryLabel': category_label,
        'Coefficient': nonzero_coefs[var]
    })

# Display results with formatted CategoryLabel and wider Coefficient display
result_df = pd.DataFrame(results_data)
pd.set_option('display.max_rows', None)
print(f"Lasso found {len(result_df)} significant variables (nonzero coefficients):")

# Prepare display copy: truncate long category labels and format coefficients for readability
max_cat_len = 40  # max characters to show for category labels
result_df['CategoryLabel'] = result_df['CategoryLabel'].astype(str).apply(
    lambda s: s if len(s) <= max_cat_len else s[:max_cat_len-3] + '...'
)

# Keep numeric coefficient for downstream use, but create a formatted display version
result_df['Coefficient'] = result_df['Coefficient'].astype(float)
result_df_display = result_df.copy()
result_df_display['Coefficient'] = result_df_display['Coefficient'].map(lambda v: f"{v: .6f}")

# Show a compact, clearly formatted table
display_cols = ['Variable', 'Label', 'Category', 'CategoryLabel', 'Coefficient']
pd.set_option('display.max_colwidth', 50)
display(result_df_display[display_cols])

Lasso found 160 significant variables (nonzero coefficients):


,Variable,Label,Category,CategoryLabel,Coefficient
0,sf12mcs_dv,SF-12 Mental Component Summary (MCS),,,-3.498345
1,finnow_5,Subjective financial situation - current,5,Finding it very difficult,1.884265
2,sclfsato_7,Satisfaction with life overall,7,completely satisfied,-1.620461
3,sclfsato_6,Satisfaction with life overall,6,mostly satisfied,-1.125316
4,sf12pcs_dv,SF-12 Physical Component Summary (PCS),,,-1.009862
5,finnow_4,Subjective financial situation - current,4,Finding it quite difficult,0.881343
6,sclfsato_3,Satisfaction with life overall,3,somewhat dissatisfied,0.826083
7,sclfsato_5,Satisfaction with life overall,5,somewhat satisfied,-0.570174
8,finfut_2,Subjective financial situation - future,2,Worse of than now,0.393240
9,sex_2,Sex,2,female,0.356643


**Very interesting outcome! Here are my key takeaways:**

Already included in my refined variable list:

- Neighbourhood social cohesion (nbrsnci_dv) is confirmed to be correlated with subjective well-being (scghq1_dv), as expected
- Demographics are important (e.g. age, gender)
- Education is important (highest qualification)
- Current economic activity is important

Variables to consider adding:
- Subjective financial situation (finnow) (finfut) -- clearly subjective financial strain is a key predictor of well-being


Variables I could consider deleting from my refined variable list:
- Job industry (not present here, and will be annoying due to lots of dummies)

Noteworthy variables:
- sclfsat* variables (e.g. satisfaction with life, health, income) are strong predictors of wellbeing. But they are effecitvely alternative wellbeing outcomes. If we include them, we risk circularity (i.e. we are predicting wellbeing with other measures of wellbeing). So I will exclude them
- sf12mcs (SF-12 mental health score) and sf12pcs (SF-12 physical health score) are strong predictors of wellbeing. They are also arguably alternative measures of wellbeing, so if we include them we risk circularity (over-control).




